In [24]:
import pandas as pd
import numpy as np
import itertools

In [25]:
#statsDF - initial stats dataframe
#gamesDF - initial games dataframe
#newGamesDF - formatted games dataframe with game date as index and rest of record in one row
#finalStats - formatted stats dataframe with player's name as the index
#mainDF: The dataframe where the main analysis will take place.

In [26]:
#STATS CELL: This prepares the stats section into a dataframe. Players should always be on even rows
#Passing in csv file and getting rid of the unused rows
statsDF = pd.read_csv('stats.csv', delim_whitespace=True)
statsDF.columns = [col.replace(',', '') for col in statsDF.columns]
# if you want to operate on multiple columns, put them in a list like so:
cols = ['PLAYER', 'MIN', 'FGM','FGA', 'FG%', '3PM', '3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL','BLK','PF','PTS','+/-']
teams = ['BOS:','CLE:','HOU:','GSW:','MEM:','NOP:','DAL:','CHA:', 'DET:','MIA:','PHI:','WAS:','ORL:', 'MIL:','ATL:' ]
teams_full = ['Boston','Cleveland','Houston','Golden', 'Detroit', 'Dallas', 'Brooklyn','Miami', 'Milwaukee', 'Indiana','Orlando', 'Memphis','Washington', 'Atlanta']
# pass them to df.replace(), specifying each char and it's replacement:
statsDF[cols] = statsDF[cols].replace({'\$': '', ',': ''}, regex=True)
#Remove columns for team name, totals, and the word 'player'
statsDF = statsDF[statsDF.PLAYER != 'PLAYER']
statsDF = statsDF[statsDF.PLAYER != 'Totals:']
statsDF = statsDF[statsDF.PLAYER != 'INACTIVE']
for i in teams:
    statsDF = statsDF[statsDF.PLAYER != i]
for i in teams_full:
    statsDF = statsDF[statsDF.PLAYER != i]
#Reset Index
statsDF = statsDF.reset_index(drop=True)

In [27]:
#GAMES CELL: This prepares the games section into a dataframe
#Passing in csv file and getting rid of the unused rows
gamesDF = pd.read_csv('games.csv', header= None, names=['a','b','c','d','e','f','g','h','i'], index_col=False, error_bad_lines=False, delim_whitespace=True)
gamesDF.columns = [col.replace(',', '') for col in gamesDF.columns]
#if you want to operate on multiple columns, put them in a list like so:
games_cols = ['a','b','c','d','e','f','g','h','i']
# pass them to df.replace(), specifying each char and it's replacement:
gamesDF[games_cols] = gamesDF[games_cols].replace({'\$': '', ',': ''}, regex=True)
# This will add the winner to column b of each 26th row
for index, row in gamesDF.iterrows():
    if index % 26 == 0 or index == 0:
        if int(gamesDF['a'].iloc[index+4]) > int(gamesDF['a'].iloc[index + 8]):
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 1]
        else:
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 5]
    else:
        pass

In [28]:
awayTeam = []
for index, row in gamesDF.iterrows():
    if index == 0 or index % 26 == 0:
        awayTeam.append(gamesDF.iloc[index+11,0])
awayTeam

['BOS', 'HOU', 'BOS', 'HOU', 'CHA', 'BKN', 'MIA', 'PHI', 'MIL', 'NOP', 'ATL']

In [29]:
#This cell will arrange the gamesDF into the actual formatted games dataframe, newGamesDF
#Setting up the index on the main stats dataframe
dateIndex = []
month = ['OCT', 'NOV', 'DEC', 'JAN', 'FEB']
for index, row in gamesDF.iterrows():
    for i in month:
        if gamesDF.iloc[index][0] == i:
            dateIndex.append(gamesDF.iloc[9,0] + " " + gamesDF.iloc[9,1][1] + ", " + gamesDF.iloc[9,2])
#The next lines will pull the data from the cells in the gamesDF and into the newGamesDF dataframe
gamesData = []
games = 0
for index, row in gamesDF.iterrows():
    if index == 0 or index % 26 == 0:
        gamesData.append(gamesDF.iloc[index+11,0])
        gamesData.append(gamesDF.iloc[index+12,0])
        gamesData.append(gamesDF.iloc[index,1])
        gamesData.append(gamesDF.iloc[index+11,1])
        gamesData.append(gamesDF.iloc[index+11,2])
        gamesData.append(gamesDF.iloc[index+11,3])
        gamesData.append(gamesDF.iloc[index+11,4])
        gamesData.append(gamesDF.iloc[index+12,1])
        gamesData.append(gamesDF.iloc[index+12,2])
        gamesData.append(gamesDF.iloc[index+12,3])
        gamesData.append(gamesDF.iloc[index+12,4])
        gamesData.append(gamesDF.iloc[index+8,0])
        gamesData.append(gamesDF.iloc[index+4,0])
        gamesData.append(gamesDF.iloc[index+22,2])
        gamesData.append(gamesDF.iloc[index+22,4])
        gamesData.append(gamesDF.iloc[index+22,6])
        games += 1
        
gamesData = np.array(gamesData)
gamesData = gamesData.reshape(games,16)
#Create an empty dataframe, based on the games date as the index
newGamesDF = pd.DataFrame(data = gamesData, columns=['Home','Away','W/L','1st Qtr H','2nd Qtr H','3rd Qtr H','4th Qtr H','1st Qtr A','2nd Qtr A','3rd Qtr A','4th Qtr A','Total H','Total A', 'Ref1','Ref2','Ref3'])
newGamesDF

,Home,Away,W/L,1st Qtr H,2nd Qtr H,3rd Qtr H,4th Qtr H,1st Qtr A,2nd Qtr A,3rd Qtr A,4th Qtr A,Total H,Total A,Ref1,Ref2,Ref3
0,BOS,CLE,CLEVELAND,19,19,33,28,29,25,18,30,102,99,"McCutchen""""","Smith""""",Forte
1,HOU,GSW,HOUSTON,34,28,26,34,35,36,30,20,121,122,"Foster""""","Garretson""""",Maddox
2,BOS,CLE,CLEVELAND,19,19,33,28,29,25,18,30,102,99,"McCutchen""""","Smith""""",Forte
3,HOU,GSW,HOUSTON,34,28,26,34,35,36,30,20,121,122,"Foster""""","Garretson""""",Maddox
4,CHA,DET,DETROIT,27,18,25,20,29,27,26,20,102,90,"DeRosa""""","Davis""""",Boland
5,BKN,IND,INDIANA,30,33,35,33,29,36,41,34,140,131,"Brothers""""","Collins""""",Cutler
6,MIA,ORL,ORLANDO,37,18,23,31,32,26,30,28,116,109,"Malloy""""","Lane""""",Holtkamp
7,PHI,WAS,WASHINGTON,23,36,23,33,32,24,34,30,120,115,"Zarba""""","Guthrie""""",Taylor
8,MIL,BOS,MILWAUKEE,24,34,18,32,26,27,27,20,100,108,"Washington""""","Lindsay""""",Goldenberg
9,NOP,MEM,MEMPHIS,34,18,18,21,27,27,25,24,103,91,"Corbin""""","Zielinski""""",Twardoski


In [30]:
#Setting up the index on the main stats dataframe
mainDFIndex = []
for index, row in statsDF.iterrows():
    if index % 2 == 0:
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
#print(mainDFIndex)

In [31]:
#Setup dicts with the sub categories
newDict = dict()
something = statsDF.T.to_dict().values()
for index, i in enumerate(something):
    if index % 2 != 0:
        newDict[index] = i

In [32]:
#put them in dataframe, index it with the main index, drop nan columns, rename the columns to their proper categories
finalStats = pd.DataFrame.from_dict(newDict,orient='index')
finalStats.index = mainDFIndex
finalStats.drop(['+/-'], axis=1, inplace=True)
finalStats = finalStats.rename(columns={'PLAYER':'MIN','MIN':'FGM', 'FGM':'FGA', 'FGA':'FG%', 'FG%':'3PM', '3PM':'3PA', '3PA':'3P%', '3P%':'FTM', 'FTM':'FTA', 'FTA':'FT%', 'FT%':'OREB', 'OREB':'DREB', 'DREB':'REB', 'REB':'AST', 'AST':'TOV', 'TOV':'BLK', 'BLK':'PF', 'PF':'PTS', 'PTS':'+/-'})
#This will replace the nan's with DNP
finalStats = finalStats.fillna('DNP')

In [33]:
player_count = len(finalStats.index)
finalStats

,AST,BLK,FG%,3PM,MIN,TOV,REB,FTA,3PA,FGM,FT%,FGA,3P%,+/-,STL,PTS,PF,OREB,DREB,FTM
Gordon Hayward,0,0,50.0,0,5:15,0,1,0,1,1,0.0,2,0.0,3,0,2,1,0,1,0
Jayson Tatum,3,0,41.6,1,36:32,1,10,3,2,5,100,12,50.0,6,0,14,4,4,6,3
Al Horford,5,0,28.5,0,32:07,0,7,7,2,2,71.3,7,0.0,8,1,9,2,0,7,5
Jaylen Brown,0,2,47.7,2,39:36,3,6,2,9,11,50.0,23,22.2,-5,0,25,5,1,5,1
Kyrie Irving,10,3,47.0,4,39:21,2,4,2,9,8,100,17,44.4,-1,0,22,4,2,2,2
Marcus Smart,3,2,31.3,0,35:03,2,9,3,4,5,66.7,16,0.0,-8,2,12,2,0,9,2
Aron Baynes,1,0,100,0,19:06,2,5,4,0,2,50.0,2,0.0,-14,1,6,5,2,3,2
Semi Ojeleye,0,0,0.0,0,8:39,0,0,0,1,0,0.0,2,0.0,-10,0,0,1,0,0,0
Terry Rozier,2,4,33.3,1,19:32,0,3,4,3,2,100,6,33.3,4,0,9,0,0,3,4
Shane Larkin,0,0,0.0,0,4:49,0,1,0,1,0,0.0,1,0.0,2,0,0,0,0,1,0


In [63]:
#STOPPED HERE. NEED TO FIX

#teams_list will hold the team name for every instance of the team and will be the team index for every player
teams_list = []
#dates list will hold the dates for every instance 
dates_list = []
#The team flag will let the for loop below choose the proper team to assign to each player
team_flag = 0
date_flag = 0
#Because multiple DNP can occur per team, a general flag is needed to tell when the DNP stop. 
flag = 0
counter = 0
#teams variable will bring the teams into a list of lists, which the next line will flatten into one list.
for x in finalStats.iterrows():
    if x[1][0] == 'DNP':
        flag = 1
        teams_list.append(teams[team_flag])
        dates_list.append(dateIndex[date_flag])
    else:
        if flag == 1:
            if x[1][0] == 'DNP':
                flag = 1
                teams_list.append(teams[team_flag])
                dates_list.append(dateIndex[date_flag])
            else:
                flag = 0
                team_flag += 1
            if team_flag % 2 == 0:
                date_flag += 1
                teams_list.append(teams[team_flag])
                dates_list.append(dateIndex[date_flag])
            else: 
                dates_list.append(dateIndex[date_flag])
                teams_list.append(teams[team_flag])
        else:
            teams_list.append(teams[team_flag])
            dates_list.append(dateIndex[date_flag])
    
#multiDFIndex will create a tuple for the multi-index dataframe              
multiDFIndex = [dates_list,teams_list,mainDFIndex]

IndexError: list index out of range

In [57]:
len(multiDFIndex[2])

284

In [40]:
#mainDF: The dataframe where the main analysis will take place.
mainDF = finalStats.set_index(multiDFIndex)

ValueError: all arrays must be same length

In [ ]:
#This will take logic about the game itself and add it to the main DataFrame. There should be field for team points,home/away, win/loss

total_points = []
win_loss = []
home_away = []

for index, row in mainDF.iterrows():
    if index[1] == newGamesDF['Away'].iloc[0]:
        total_points.append(newGamesDF['Total A'].iloc[0])
        home_away.append('Away')
    else:
        total_points.append(newGamesDF['Total H'].iloc[0])
        home_away.append('Home')
        
for index, row in mainDF.iterrows():
    if index[1] == newGamesDF['W/L'].iloc[0]:
        win_loss.append('Win')
    else:
        win_loss.append('Loss')
mainDF['Total Points'] = total_points
mainDF['Home/Away'] = home_away
mainDF['W/L'] = win_loss
mainDF.apply(pd.to_numeric, errors=0)

In [ ]:
#This function will calculate the projected fantasy points per game
def fantasy_points(row):
    try:
        score = (int(row['3PM']) * 3) + (int(row['FGM']) * 2) + (int(row['REB']) * 1) + (int(row['AST']) * 1.5) + (int(row['BLK']) * 3) + (int(row['STL']) * 3) + (int(row['TOV']) * -1)
        return score
    except:
        pass

mainDF['Fantasy Score'] = mainDF.apply(fantasy_points, axis=1)